## Monads
- Monads are a way of defining computation
- A monad is a container type m along with two functions:

  - return :: a -> m a
  - bind :: m a -> (a-> m b) -> m b
  - In Haskell, bind is written as **>>=**
  
  
- These functions must obey three laws:

  - **Left identity** return a >>= f is the same as f a 
  - **Right identity** m >>= return is the same as m
  - **Associativity** (m >>= f) >>= g is the same as m >>= (\x -> f x >>= g)

### Understanding Return
- return :: a -> m a
- the return keyword takes an element and puts it into a monad
- this is a one-way trip
- very much like pure in the applicative type class

In [ ]:
instance Monad Maybe where
    return a = Just a
instance Monad [] where
    return a = [a]
instance Monad (Either a) where
    return a = Right a

### Understanding Bind
- All the magic happens in bind
- bind :: m a -> (a -> m b) -> m b
  - the first argument is a monad
  - the second argument takes monad, unpack it, and repackages it with the help of the function argument
  
    - exactly how it does that is the magic part

In [1]:
-- Bind for maybe
Nothing  >>= f = Nothing
(Just a) >>= f = f a

-- remember that f returns a monad

#### Motivation
<img src="motiv.png" alt="motiv" width="500"/>
<img src="diff.png" alt="diff" width="500"/>

### A Calculator with Monads

In [2]:
-- normal implementation without monad
minc x = x >>= (\xx -> return (xx + 1))
madd a b = a >>= (\aa ->
           b >>= (\bb -> return (aa + bb)))

- The above code works, but here's a better way
- First define functions list to convert a function to monadioc form for us

**These are part of Control.Monad**:

     liftM f a = a >>= (\aa -> return (f aa))
     liftM2 f a b = a >>= (\aa ->
                    b >>= (\bb -> return (f aa bb)))

Thus we can use lifting to implement the calculator:

In [31]:
inc :: Integer -> Integer
inc x = x + 1

add :: Integer -> Integer -> Integer
add x y =  x + y

plus :: Num a => a -> a -> a
plus a b = a + b

sub :: Integer -> Integer -> Integer
sub x y =  x - y

In [33]:
import Control.Monad

minc = liftM inc
madd = liftM2 add
msub = liftM2 sub
mdiv a b = a >>= (\aa->
           b >>= (\bb->
            if bb == 0 then fail "\0"
                       else return (aa `div` bb)))

Line 3: Use fmap
Found:
liftM
Why not:
fmap

#### fail
- fail is another useful monadic function, defined in the MonadFail typeclass
- here it's defined as Nothing

### The Maybe Monad
complete definition for maybe monad:

In [51]:
import Control.Monad
import qualified Control.Monad.Fail as Fail

instance Monad Maybe where
    return = Just
    
    (>>=) Nothing f = Nothing
    (>>=) (Just a) f = f a
    
    fail s = Nothing

: 

In [42]:
minc (Just 10)
madd (minc (Just 10)) (Just 20)
mdiv (minc (Just 10)) (minc (Just 2))
minc (mdiv (minc (Just 10)) (minc (Just 2)))
minc (mdiv (minc (Just 10)) (Just 0))

Just 11

Just 31

Just 3

Just 4

Nothing

### The List Monad

List can be monads too. The trick is deciding what bind should do

In [43]:
instance Monad [] where
    return a = [a]
    
    (>>=) [] f = []
    (>>=) xs f = concatMap f xs
    
    fail s = Nothing

-- note that we havent change anything on our lifted calculator example

: 

In [54]:
minc [1,2,3]
madd [1,2,3] [10,200]

[2,3,4]

[11,201,12,202,13,203]

## The State Monads